In [1]:
import pandas as pd
import re
import ast
import json
from collections import Counter
from dateutil import parser
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
import numpy as np
import random

pd.options.mode.chained_assignment = "warn"

from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_csv('drive/MyDrive/Colab Notebooks/news_before_processing.csv',  sep=';')

In [ ]:
df

,url,date and time,tag,title,text
0,https://www.kommersant.ru/doc/973901,"22.04.2004, 17:39",Мир,\r\n Абашидзе упразднил МГБ Аджарии\r\n...,Глава Аджарии Аслан Абашидзе принял решение об...
1,https://www.kommersant.ru/doc/2200895,"30.05.2013, 18:15",Общество,\r\n РЖД будут продавать билеты через «...,С 1 июля Российские железные дороги начнут про...
2,https://www.kommersant.ru/doc/3859122,"18.01.2019, 20:48",Финансы,\r\n Объем операций в биткойнах...,Использование биткойнов для оплаты товаров и у...
3,https://www.kommersant.ru/doc/5347136,"09.05.2022, 20:55",Смерть Елизаветы II,\r\n Королева Елизавета II проп...,Королева Великобритании Елизавета II не будет ...
4,https://www.kommersant.ru/doc/1171617,"17.05.2009, 19:39",Общество,\r\n Япония и Великобритания сообщили о...,Число официально подтвержденных случаев зараже...
...,...,...,...,...,...
1460939,https://www.fontanka.ru/2023/11/17/72922160/,2023.11.16,['Академия художеств'],В музее Академии художеств вместе покажут Нико...,На выставку «Возвращение. Николай Фешин и Степ...
1460940,https://www.fontanka.ru/2023/11/17/72921161/,2023.11.17,"['Куда сходить бесплатно', 'Афиша Петербурга']",Куда пойти 17–19 ноября: Османская империя в Э...,В предпоследний уик-энд ноября несколько десят...
1460941,https://www.fontanka.ru/2023/11/17/72921161/,2023.11.17,"['Куда сходить бесплатно', 'Афиша Петербурга']",Куда пойти 17–19 ноября: Османская империя в Э...,В предпоследний уик-энд ноября несколько десят...
1460963,https://www.fontanka.ru/2023/11/17/72923615/,2023.11.17,"['Развлечения', 'Музыка', 'Тест', 'Песни', 'Юм...",Все неправильно слышали слова этих песен. А чт...,Очень много воспоминаний у нас с вами оставили...


###Processing Data


In [19]:
#processing
df = df.drop_duplicates()
df = df.drop(df[df['tag'].apply(lambda x: x == '[]')].index)
df.dropna(inplace=True)

def get_tags(tag_string):
    try:
        tags_list = ast.literal_eval(tag_string)
        if isinstance(tags_list, list):
            if len(tags_list) == 1:
                return tags_list[0]
            else:
                return tag_string
    except:
        return tag_string

    if tag_string == '"Болотное дело"':
        return 'Болотное дело'


# первичная обработка тэгов
df['tags_list'] = df['tag'].apply(get_tags)

def clean_text(text):
    text = str(text)
    text = re.sub(r'^[\r\n\s]+|[\r\n\s]+$', '', text)
    return text.strip()

df['tags_list'] = df['tags_list'].apply(clean_text)

#считаем распространенность тэгов
tags_list = []

for cell in df['tags_list']:
    if '[' in cell:
        tags = [tag.strip().replace("'", "") for tag in cell[1:-1].split(',')]
        for tag in tags:
            tags_list.append(tag.strip())
    else:
        tags_list.append(cell.strip())

tag_counter = Counter(tags_list)

#вторичная обработка тэгов (выбор одного популярного тэга, если представлено несколько)
def select_most_popular_tag(text):
    if re.match(r"\['.*'\]$", text):
        tags = re.findall(r"'([^']+)'", text)
        valid_tags = [tag.strip() for tag in tags if tag_counter.get(tag.strip())]
        return max(valid_tags, key=tag_counter.get)
    else:
        return text

df['most_popular_tag'] = df['tags_list'].apply(select_most_popular_tag)

df.drop('tags_list', axis=1, inplace=True)

In [20]:
#Удаление строк, в которых содержится очень редкий тэг
most_popular_tag_counter = Counter(df['most_popular_tag'])
tags_to_remove = [tag for tag, count in most_popular_tag_counter.items() if count < 100]
df = df[~df['most_popular_tag'].isin(tags_to_remove)]

In [21]:
#преобразование даты
df = df[df['date and time'] != '01.01.0001, 00:00']

month_dict = {
    'января': '01','февраля': '02','марта': '03','апреля': '04','мая': '05',
    'июня': '06','июля': '07','августа': '08','сентября': '09','октября': '10',
    'ноября': '11','декабря': '12'
}

df['date and time'].replace(month_dict, regex=True, inplace=True)

df['date and time'] = df['date and time'].apply(lambda x: parser.parse(x, fuzzy=True) if pd.notna(x) else x)

df['date and time'] = pd.to_datetime(df['date and time'], errors='coerce')

print("Количество неправильных дат:", df['date and time'].isna().sum())

df = df[df['date and time'].dt.year >= 2001]

Количество неправильных дат: 0


In [22]:
#преобразование title, text
def clean_text(text):
    text = str(text)
    text = re.sub(r'^[\r\n\s]+|[\r\n\s]+$', '', text)
    return text.strip()

for col in tqdm(['title', 'text']):
    df[col] = df[col].apply(clean_text)

df = df[df['text'].str.strip() != '']

100%|██████████| 2/2 [01:50<00:00, 55.31s/it]


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1271195 entries, 0 to 1460901
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   url               1271195 non-null  object        
 1   date and time     1271195 non-null  datetime64[ns]
 2   tag               1271195 non-null  object        
 3   title             1271195 non-null  object        
 4   text              1271195 non-null  object        
 5   most_popular_tag  1271195 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 67.9+ MB


In [37]:
df.to_csv('news_after_processing.csv', index=False, sep=';')

###Processing text and Base Modelling

In [2]:
df = pd.read_csv('drive/MyDrive/Colab Notebooks/news_after_processing.csv',  sep=';')

In [37]:
df.head(2)

,url,date and time,tag,title,text,most_popular_tag
0,https://www.kommersant.ru/doc/973901,2004-04-22 17:39:00,Мир,Абашидзе упразднил МГБ Аджарии,Глава Аджарии Аслан Абашидзе принял решение об упразднении министерства госбезопасности автономии.,Мир
1,https://www.kommersant.ru/doc/2200895,2013-05-30 18:15:00,Общество,РЖД будут продавать билеты через «Почту России»,"С 1 июля Российские железные дороги начнут продавать билеты в офисах «Почты России», заявил на пресс-конференции в Петербурге заместитель начальника департамента пассажирских сообщений ОАО «РЖД» Дмитрий Корней. Проект коснется не всех городов, отмечает «Фонтанка». «Там, где в связи с малыми объемами продаж кассы уже не работают, в отделениях почтовой связи можно будет приобрести электронный билет. И, не распечатывая бланк заказа, не проходя процедуру так называемой регистрации, сесть в поезд. При себе пассажир должен будет иметь только бланк, который выдадут на почте», — уточнил первый замначальника Северо-Западного филиала ОАО «Федеральная пассажирская компания» Станислав Зотин. Всего проект стартует в 24 городах.\nКак пытаются улучшить работу «Почту России» — читайте в материале «Ъ» «Большие перемены». Об инновациях в РЖД читайте в статье «Ъ» «Чемодан—вокзал—досмотр».",Общество


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1271195 entries, 0 to 1271194
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   url               1271195 non-null  object
 1   date and time     1271195 non-null  object
 2   tag               1271195 non-null  object
 3   title             1271195 non-null  object
 4   text              1271195 non-null  object
 5   most_popular_tag  1271195 non-null  object
dtypes: object(6)
memory usage: 58.2+ MB


In [20]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.most_popular_tag, test_size=0.2, random_state=42)
x_train

1108254    Россияне в скором времени ...
17579      В 2022 году мировые цены н...
890062     [Россиянам из Салехарда (Я...
957855     [Министры иностранных дел ...
1047296    Спикер Госдумы Вячеслав Во...
                       ...              
110268     15 сентября следствию удал...
259178     [Госдепартамент США заявил...
131932     [Президент Афганистана Хам...
671155     [В Австралии прооперирован...
121958     [Вакцина от коронавируса н...
Name: text, Length: 1016956, dtype: object

In [31]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [34]:
data = pd.DataFrame(columns=['Parameters', 'Accuracy', 'Precision', 'Recall', 'F1-score'])

In [35]:
#Стохастический градиентный спуск с log-loss и TfidfVectorizer
label_encoder = LabelEncoder()
y_train_mlb = label_encoder.fit_transform(y_train)
y_test_mlb = label_encoder.transform(y_test)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 1))),#tokenizer=word_tokenize
    ('clf', SGDClassifier(max_iter=100000, loss='log_loss'))
])

ran = np.arange(len(x_train))
inds = np.array_split(ran, 30)

for chunk in tqdm(inds):
    chunk_with_random = np.concatenate((chunk, np.array(random.sample(list(ran), k=1000))))
    pipeline.fit(np.array(x_train)[chunk_with_random], y_train_mlb[chunk_with_random])

predicted_labels = pipeline.predict(x_test)

predicted_labels = pipeline.predict(x_test)
accuracy = accuracy_score(y_test_mlb, predicted_labels)
precision = precision_score(y_test_mlb, predicted_labels, average='weighted')
recall = recall_score(y_test_mlb, predicted_labels, average='weighted')
f1 = f1_score(y_test_mlb, predicted_labels, average='weighted')

params = str(pipeline.named_steps['tfidf'].get_params()) + str(pipeline.named_steps['clf'].get_params())
data = data.append({'Parameters': params, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}, ignore_index=True)

100%|██████████| 30/30 [31:48<00:00, 63.61s/it]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-35-b2cf7a8e74d9>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Parameters': params, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}, ignore_index=True)


In [36]:
pd.set_option('display.max_colwidth', None)
data

,Parameters,Accuracy,Precision,Recall,F1-score
0,"{'analyzer': 'word', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 1, 'ngram_range': (1, 1), 'norm': 'l2', 'preprocessor': None, 'smooth_idf': True, 'stop_words': None, 'strip_accents': None, 'sublinear_tf': False, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'use_idf': True, 'vocabulary': None}{'alpha': 0.0001, 'average': False, 'class_weight': None, 'early_stopping': False, 'epsilon': 0.1, 'eta0': 0.0, 'fit_intercept': True, 'l1_ratio': 0.15, 'learning_rate': 'optimal', 'loss': 'log_loss', 'max_iter': 100000, 'n_iter_no_change': 5, 'n_jobs': None, 'penalty': 'l2', 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'tol': 0.001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}",0.551312,0.573739,0.551312,0.486727


In [ ]:
#Вариант предобработки текста
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('russian'))
    tokens = [word for word in tokens if word not in stop_words]

    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text


df['preprocessed_text'] = df['text'].apply(preprocess_text)